In [ ]:
#!/usr/bin/env python
import actionlib
from fetch_control import *
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from control_msgs.msg import FollowJointTrajectoryGoal, JointTolerance, FollowJointTrajectoryAction
import rospy
import copy
import pickle
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion, Twist
import sys
from gazebo_msgs.srv import *
from geometry_msgs.msg import *
from gazebo_msgs.msg import * 
import time
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from math import sin, cos
from openravepy import *
import numpy as np
from nav_msgs.msg import Odometry
from moveit_msgs.msg import MoveItErrorCodes
from moveit_python import MoveGroupInterface, PlanningSceneInterface
import moveit_commander
import sys
import roslib
import tf

rospy.init_node("physical_replan_execution")

In [ ]:
# move the robot to goal pose
def execute_base_movement(goal_pose):
    '''
    goal_pose: [x, y, theta]
    '''
    client = actionlib.SimpleActionClient("move_base", MoveBaseAction)
    print("Waiting for move_base...")
    client.wait_for_server()
    # print("goal position: ", goal_pose)
    x = goal_pose[0]
    y = goal_pose[1]
    theta = goal_pose[2]
    print("goal position: ", x, y, theta)
    
    frame = "map"
    orientation = quatFromAxisAngle(np.asarray([0.,0.,theta]))

    move_goal = MoveBaseGoal()
    move_goal.target_pose.pose.position.x = x
    move_goal.target_pose.pose.position.y = y
    move_goal.target_pose.pose.orientation.x = orientation[1]
    move_goal.target_pose.pose.orientation.y = orientation[2]
    move_goal.target_pose.pose.orientation.z = orientation[3]
    move_goal.target_pose.pose.orientation.w = orientation[0]
    # move_goal.target_pose.pose.orientation.z = sin(theta / 2.0)
    # move_goal.target_pose.pose.orientation.w = cos(theta / 2.0)
    move_goal.target_pose.header.frame_id = frame
    move_goal.target_pose.header.stamp = rospy.Time.now()

    client.send_goal(move_goal)
    result = client.wait_for_result()
    print(result)
    print("reached!")

In [ ]:
traj_dump = open("ros_trajectory_v3.pickle", "rb")
trajectories = pickle.load(traj_dump)
traj_dump.close()
values = trajectories[0] # execute first base trajectory (moving to tool shelf)
t_type = values[0]
t_val = values[1]
t_pose_end = values[2]
# t_pose_end[1] += 0.

execute_base_movement(t_pose_end)
# execute_base_movement([0,-8,0]) # Send somewhere else if needed

In [ ]:

def gms_client(model_name,relative_entity_name):
    '''
    get model state client
    retrieves the model state (pose, orientation) of the model w.r.t the relative entity
    '''
    rospy.wait_for_service('/gazebo/get_model_state')
    try:
        gms = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)
        resp1 = gms(model_name, relative_entity_name)
        return resp1
    except rospy.ServiceException, e:
        print "Service call failed: %s"%e

In [ ]:
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

def sms_client(model_name, goal_pose):
    '''
    set model in gazebo to given goal pose
    goal_pose: [x, y, theta] 
    '''

    orientation = quatFromAxisAngle(np.asarray([0.,0.,goal_pose[2]]))

    state_msg = ModelState()
    state_msg.model_name = model_name
    state_msg.pose.position.x = goal_pose[0]
    state_msg.pose.position.y = goal_pose[1]
    # state_msg.pose.position.z = pose[6]
    state_msg.pose.orientation.x = orientation[1]
    state_msg.pose.orientation.y = orientation[2]
    state_msg.pose.orientation.z = orientation[3]
    state_msg.pose.orientation.w = orientation[0]
    set_state(state_msg)

In [ ]:
# Execute to teleport fetch to goal pose (will mess up localisation) use for debugging 
sms_client('fetch', t_pose_end)

In [ ]:
# Set ply object to its original position in case it flies off
state_msg = ModelState()
state_msg.model_name = 'unit_box_0'
state_msg.pose.position.x = -0.473003#-0.462124 #-0.600290
state_msg.pose.position.y = -10.395247#-10.485400 #-10.323116
state_msg.pose.position.z = 0.609278#0.614564 #0.615354
quaternion = tf.transformations.quaternion_from_euler(0, 0, 0)
state_msg.pose.orientation.x = quaternion[0]
state_msg.pose.orientation.y = quaternion[1]
state_msg.pose.orientation.z = quaternion[2]
state_msg.pose.orientation.w = quaternion[3]
set_state(state_msg)

In [1]:
# Start openrave simulation
import sys
sys.path.append('/home/abhyudaya/Desktop/TMP_Merged/')

from openravepy import *
import numpy as np
import random
import src.OpenraveUtils as OpenraveUtils
from src.DataStructures.Generator import Generator
from src.Simulators.OpenRaveSimulator import *
import math
import random
from trac_ik_python.trac_ik import IK
import Config

import numpy as np
import math
import time
import json

# sys.path.append('/home/local/ASUAD/asrinet1/AAIR/TMP_Merged/')
from src.Robots.Models import FetchOpenRaveRobotModel
import openravepy
# import model_file_generator as MFG
from tf.transformations import euler_from_quaternion, euler_from_matrix

# OpenRave_OMPL_RRT_Connect_Planner
from src.MotionPlanners.OpenRaveMotionPlanner import *
import src.util as util
from prpy.planning import ompl, CBiRRTPlanner, chomp
from openravepy import planningutils
from fetch_control import *

env = openravepy.Environment()
env.SetViewer('qtcoin')


env.Load('../worlds/env.dae')
FetchOpenRaveRobotModel(env, False)


h = None

In [ ]:
# get robot pose in gazebo
res = gms_client('fetch', 'world')
current_pose = res.pose
print("fetch")
print(current_pose.position)
q = [current_pose.orientation.w,current_pose.orientation.x, current_pose.orientation.y, current_pose.orientation.z]
angle = axisAngleFromQuat(q)
print(angle)

In [ ]:
# set robot pose in openrave from gazebo
rotation_matrix = matrixFromQuat(q)
rotation_matrix[0][3] = current_pose.position.x
rotation_matrix[1][3] = current_pose.position.y
rotation_matrix[2][3] = current_pose.position.z

robot = env.GetKinBody('fetch')
taskmanip = interfaces.TaskManipulation(robot)
taskmanip.ReleaseFingers()
robot.SetTransform(rotation_matrix)

In [ ]:
def get_fast_ik_solutions(env, robot, end_effector_transform, do_check_collisions=False):
    filter_option = openravepy.IkFilterOptions.CheckEnvCollisions
    # filter_option = None
    if not do_check_collisions:
        filter_option = openravepy.IkFilterOptions.IgnoreEndEffectorCollisions
    
    ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(robot, iktype=openravepy.IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        raveLogInfo("Generating IKModel for " + str(robot))
        ikmodel.autogenerate()
    solutions = ikmodel.manip.FindIKSolutions(end_effector_transform, filter_option)
    # DrawAxes(self.env, end_effector_transform)
    print len(solutions), " IK solutions found" 

    return solutions[:20] # return only 20 solutions

In [ ]:
def get_pick_pose(gripper_offset):
    '''
    get pose of the gripper for pick action
    gripper_offset: distance from the edge of the plank the gripper should move to
    '''

    robot = env.GetRobots()[0]
    model_name = 'fetch'
    fetch_body = env.GetKinBody(model_name)
    fetch_aabb = fetch_body.ComputeAABB()
    fetch_x_len = fetch_aabb.extents()[0]*2
    fetch_y_len = fetch_aabb.extents()[1]*2
    fetch_z_len = fetch_aabb.extents()[2]*2
    print "fetch:", fetch_x_len, fetch_y_len, fetch_z_len
    fetch_original_pose = poseFromMatrix(fetch_body.GetTransform())

    model_name = 'plank4' # 'ply_object'
    object_name = model_name
    body = env.GetKinBody(model_name)
    body_pose = poseFromMatrix(body.GetTransform())
    body_aabb = body.ComputeAABB()
    body_x_len = body_aabb.extents()[0]*2
    body_y_len = body_aabb.extents()[1]*2
    body_z_len = body_aabb.extents()[2]*2
    body_center_x = body_aabb.pos()[0]
    body_center_y = body_aabb.pos()[1]
    print "target:", body_center_x, body_center_y
    print body_x_len, body_y_len, body_z_len

    world_T_machine = body.GetTransform()


    # pick object
    z_rot_angle = -np.pi / 2
    x_rot_angle = np.pi / 2
    x_offset = 0 # body_x_len / 2
    y_offset = body_y_len / 2 + gripper_offset # 0.15 + gripper_offset #
    z_offset = 0

    #ply object
    z_rotation_matrix = np.identity(4)
    z_rotation_matrix[0][0] = math.cos(z_rot_angle)
    z_rotation_matrix[0][1] = -(math.sin(z_rot_angle))
    z_rotation_matrix[1][0] = math.sin(z_rot_angle)
    z_rotation_matrix[1][1] = math.cos(z_rot_angle)

    x_rotation_matrix = np.identity(4)
    x_rotation_matrix[1][1] = math.cos(x_rot_angle)
    x_rotation_matrix[2][1] = math.sin(x_rot_angle)
    x_rotation_matrix[1][2] = -math.sin(x_rot_angle)
    x_rotation_matrix[2][2] = math.cos(x_rot_angle)

    # ply_object
    z_rotation_matrix[0][3] = x_offset
    z_rotation_matrix[1][3] = y_offset
    z_rotation_matrix[2][3] = z_offset

    world_T_robot = np.matmul(world_T_machine, np.matmul(z_rotation_matrix, x_rotation_matrix))

    pose = poseFromMatrix(world_T_robot)
    h = openravepy.misc.DrawAxes(env, pose)
    return h, pose, world_T_robot

In [ ]:
def planToConfiguration(goal_config):
    '''
    find motion plan to given IK
    goal_config: IK to move arm to
    '''

    global  robot
    planner = ompl.OMPLPlanner('RRTConnect')
    simplifier = ompl.OMPLSimplifier()
    # Motion Planning to reach joint state value(s)
    # Get trajectory from planner based on type of goal config passed
    # ( config a.k.a ik solutions a.k.a joint states )
    try:
        # if len(goal_configs) == 1:
            # If goal is a single IK solution, then call PlanToConfiguration
        trajectory_object = planner.PlanToConfiguration(robot, goal_config)
        # else:
            # If goal is a list of IK solutions, then call PlanToConfigurations
            # trajectory_object = planner.PlanToConfigurations(robot, goal_configs)
        if hasattr(planner, 'default_ompl_args'):
            # print("simplifying..")
            # If planner is from OMPL, then simplify the trajectory
            trajectory_object = simplifier.ShortcutPath(robot,trajectory_object)
    except Exception as e:#PlanningError:
        print("Exception ", e)
        # print(__file__.split('/')[-1],": Could not find motion plan")
        return None
    # Retime and serialize the trajectory
    _ = planningutils.RetimeTrajectory(trajectory_object)
    trajectory_object = trajectory_object.serialize()
    return trajectory_object

In [ ]:
def execute_manip_trajectory(ros_traj):
    '''
    execute the trajectory in ROS/Gazebo
    ros_traj: deserialized trajectory object returned from the planner
    '''
    client = actionlib.SimpleActionClient('/arm_with_torso_controller/follow_joint_trajectory',
                                            FollowJointTrajectoryAction)
    print("waiting for client..")
    client.wait_for_server()
    
    id_gen = actionlib.GoalIDGenerator()
    jtg = FollowJointTrajectoryGoal()
    jtg.trajectory = ros_traj
    print("sending trajectory")
    client.send_goal(jtg)
    client.wait_for_result()
    print("done")

In [ ]:
# moveit_commander interfaces with gazebo arm
import moveit_commander
group_name = "arm_with_torso"
group = moveit_commander.MoveGroupCommander(group_name)

In [ ]:
# Get robot joint values in gazebo
print(group.get_active_joints())
current_values = group.get_current_joint_values()
joint_values = {}
for i, joint_name in enumerate(group.get_active_joints()):
    joint_values[joint_name] = current_values[i]
print(joint_values)

In [ ]:
# Set gazebo joint values to openrave's robot
# since we plan in openrave
or_joints = {}
for joint_name in joint_values.keys():
    joint_index = robot.GetJointIndex(joint_name)
    print(joint_name, joint_index, joint_values[joint_name])
    or_joints[joint_index] = joint_values[joint_name]
or_dof_values = []
for joint_index in robot.GetActiveDOFIndices():
    or_dof_values.append(or_joints[joint_index])
# print(robot.GetActiveJointIndices())
print(robot.GetActiveDOFIndices())
print(or_dof_values)

robot.SetActiveDOFValues(or_dof_values)

### motion planning and execution

In [ ]:
# move to pregrasp pose
# pregrasp pose: gripper_offset = 0.2
# grasp pose: gripper_offset = 0.169

# Get the pick pose
if h is not None:
    h.Close()
h, pick_pose, pick_matrix = get_pick_pose(0.169) # pick pose with offset meters away
print(pick_matrix)

In [ ]:
# Get the IK for goal_pose
goal_pose = pick_matrix

goal_configs = get_fast_ik_solutions(env, robot, goal_pose, do_check_collisions=True)
print(len(goal_configs))

In [ ]:
# Plan from current pose to goal pose

# get current joint values
inital_robot_dofs = robot.GetDOFValues()
initial_active_dofs = robot.GetActiveDOFValues() 

# Plan for the IKs
for goal_config in goal_configs:
    i = 0
    while i < 5:
        with env:
            trajectory_object = planToConfiguration(goal_config)
        if trajectory_object is not None:
            break
        print(trajectory_object)
        i += 1
    if trajectory_object is not None:
        break
# after planning, the robot in openrave remains at the goal pose
# so reset the openrave robot back to the starting pose
robot.SetDOFValues(inital_robot_dofs)
robot.SetActiveDOFValues(initial_active_dofs)

In [ ]:
# Deserialize the trajectory for execution
des_traj = RaveCreateTrajectory(env, '')
Trajectory.deserialize(des_traj, trajectory_object)

In [ ]:
# Execute on Openrave
with robot:
    robot.GetController().SetPath(des_traj)
robot.WaitForController(0)

In [ ]:
# Convert the trajectory to ROS format
ros_traj = util.ros_trajectory_from_openrave(robot, des_traj, velocity_scale=0.1)

In [ ]:
# Execute the trajectory in Gazebo
execute_manip_trajectory(ros_traj)

### GO BACK TO PLAN FOR GRASP POSE

In [ ]:
# When robot is ready to grasp

# close the grippers in gazebo
gripper = Gripper()
gripper.close(100)

# Close gripper in openrave
# taskmanip = interfaces.TaskManipulation(robot)
# taskmanip.CloseFingers()

# Attach the object in openrave (we do not close the gripper cause openrave then thinks it's in collision with the plank)
robot.Grab(env.GetKinBody('plank4'))

### Plan to final pose

In [ ]:
initial_active_dofs = robot.GetActiveDOFValues()

In [ ]:
# init_pose = [0.15, 0.72466344, -0.05064385, -1.73952133, 2.25099986, -1.50486781, -0.02543545, 1.4] #2.76926565] # arm in front of body

# init_pose = [0.0, 1.31993457, 1.40000903, -0.200000347, 1.71990992, -1.02841391e-05, 0, 7.12985942e-06] # tucked but plank extending out

init_pose = [0.0, 1.31993457, 1.40000903, -0.200000347, 1.71990992, -1.02841391e-05, 1.3, 1.5] # tucked and plank close to body

# inital_robot_dofs = robot.GetDOFValues()
# for goal_config in goal_configs:
i = 0
while i < 5:
    with env:
        trajectory_object = planToConfiguration(init_pose)
    if trajectory_object is not None:
        break
    else:
        print("retrying..")
    # print(trajectory_object)
    i += 1
if trajectory_object is not None:
    print("found")
# robot.SetDOFValues(inital_robot_dofs)


In [ ]:
robot.SetActiveDOFValues(initial_active_dofs)

In [ ]:
# Deserialize the trajectory for execution
des_traj = RaveCreateTrajectory(env, '')
Trajectory.deserialize(des_traj, trajectory_object)

In [ ]:
# Execute on Openrave
with robot:
    robot.GetController().SetPath(des_traj)
robot.WaitForController(0)

In [ ]:
# Convert the trajectory to ROS format
ros_traj = util.ros_trajectory_from_openrave(robot, des_traj, velocity_scale=0.1)

In [ ]:
# Execute the trajectory in Gazebo
execute_manip_trajectory(ros_traj)

### Move to next machine

In [ ]:
values = trajectories[5] # Execute base navigation to move to next machine
t_type = values[0]
t_val = values[1]
t_pose_end = values[2]
# t_pose_end[1] += 0.

execute_base_movement(t_pose_end)

### OLD/TEST CODE

In [ ]:
def get_joint_values_from_waypoint(waypoint, robot):
        try:
            n = robot.GetActiveDOF()
            # print("active DOFs:", n)
        except Exception, e:
            print dir(robot)
            print e
            n = 8
        return waypoint[0:n]
        
def execute_trajectory(trajectory_object):
    traj = RaveCreateTrajectory(env, '')
    Trajectory.deserialize(traj, trajectory_object)
    
    numWayPoints = traj.GetNumWaypoints()
    lastWayPoint = traj.GetWaypoint(numWayPoints - 1)
    lastWayPointDOFs = get_joint_values_from_waypoint(lastWayPoint, robot)

    robot.SetActiveDOFValues(lastWayPointDOFs)

In [ ]:
# Use old planner (no prpy)
def get_trajectory_from_ik_solutions(solutions):
    trajectory_object = None
    for i in range(len(solutions)):
        selected_iksolution = solutions[i]

        rrt_connect_planner = OpenRave_OMPL_RRT_Connect_Planner(env, robot)
        try:
            print "fetching motion plan ", i
            trajectory_object, is_success, fail_cause = rrt_connect_planner.get_mp_trajectory_to_goal(robot, goal_transform=selected_iksolution)
            print trajectory_object, is_success, fail_cause
            break
        except Exception as e:
            print "FAIL"
    return trajectory_object

In [ ]:
# solutions = get_fast_ik_solutions(env, robot, pick_pose)
# trajectory_object = get_trajectory_from_ik_solutions(solutions)
# if trajectory_object is not None:
#     print "Trajectory found"

In [ ]:
# Execute TMP trajectory
values = trajectories[1]
t_type = values[0]
t_val = values[1]
execute_manip_trajectory(t_val)

In [ ]:
def arm_tuck():
    joints = ["shoulder_pan_joint", "shoulder_lift_joint", "upperarm_roll_joint",
                "elbow_flex_joint", "forearm_roll_joint", "wrist_flex_joint", "wrist_roll_joint"]
    pose = [1.32, 1.40, -0.2, 1.72, 0.0, 1.66, 0.0]

    move_group = MoveGroupInterface("arm", "base_link")
    while not rospy.is_shutdown():
        result = move_group.moveToJointPosition(joints, pose, 0.1)
        if result.error_code.val == MoveItErrorCodes.SUCCESS:
            return
arm_tuck()

In [ ]:
gripper.close()

In [3]:
from openravepy import *
model_name = 'pedestal'
object_name = model_name
body = env.GetKinBody(model_name)
body_pose = poseFromMatrix(body.GetTransform())
body_aabb = body.ComputeAABB()
body_x_len = body_aabb.extents()[0]*2
body_y_len = body_aabb.extents()[1]*2
body_z_len = body_aabb.extents()[2]*2
body_center_x = body_aabb.pos()[0]
body_center_y = body_aabb.pos()[1]
print "target:", body_center_x, body_center_y
print body_x_len, body_y_len, body_z_len

target: -0.45123419165611334 -10.487975120544432
0.2000000000000025 0.2000000000000064 0.14000000000001145


In [ ]:
body_x_len = body_aabb.extents()[0]
body_y_len = body_aabb.extents()[1]
body_z_len = body_aabb.extents()[2]

In [ ]:
mat = np.eye(3)
mat[0,0] = (0.1 * (body_y_len**2 + body_z_len**2))
mat[1,1] = (0.1 * (body_x_len**2 + body_z_len**2)) 
mat[2,2] = (0.1 * (body_x_len**2 + body_y_len**2))

mat[0,1] = (0.1 * (body_x_len * -body_y_len))
mat[1,0] = (0.1 * (body_x_len * -body_y_len))

mat[1,2] = (0.1 * (body_z_len * -body_y_len))
mat[2,1] = (0.1 * (body_z_len * -body_y_len))

mat[0,2] = (0.1 * (body_x_len * body_z_len))
mat[2,0] = (0.1 * (body_x_len + body_z_len))
print(mat[2][2])

In [7]:
# Create pedestal in openrave
def create_box(env, body_name, t, dims, color=[0,1,1]):
  infobox = openravepy.KinBody.GeometryInfo()
  infobox._type = openravepy.GeometryType.Box
  infobox._vGeomData = dims
  infobox._bVisible = True
  infobox._vDiffuseColor = color
  infobox._t[2, 3] = dims[2] / 2

  box = openravepy.RaveCreateKinBody(env, '')
  box.InitFromGeometries([infobox])
  box.SetName(body_name)
  box.SetTransform(t)

  return box

t = openravepy.matrixFromPose([1, 0, 0, 0, 0, 0, 0])
# t = openravepy.matrixFromPose([1, 0, 0, 0, -0.473003, -10.395247, 0.619278])
# box = create_box(env, 'plank', t, [0.157823/2, 0.168412/2 , 0.004445], [0.2, 0.2, 0.2]) #plank
box = create_box(env, 'pedestal', t, [0.2, 0.2 , 0.14], [0.4, 0.2, 0.1]) #pedestal
env.Add(box, False)

In [ ]:
t

In [ ]:
body = env.GetKinBody('fetch')
env.Remove(body)

In [ ]:
env.Save('/home/abhyudaya/catkin_ws/src/planning/worlds/env.dae')

In [6]:
# Run `rosrun gazebo_ros gazebo` before execution

import rospy, tf
from gazebo_msgs.srv import DeleteModel, SpawnModel, SetModelState
from geometry_msgs.msg import *
from gazebo_msgs.msg import ModelState 
import json
import openravepy
# import Config
# ENV_GEN_DIR = Config.TEST_DIR + 'EnvGenerators/' + Config.TASK + '/'


env = openravepy.Environment()
env.SetViewer('qtcoin')
# env.Load('../worlds/env.dae')
openravepy.misc.DrawAxes(env, [1,0,0,0,0,0,0])

In [2]:
dae15 = '/home/abhyudaya/catkin_ws/src/planning/worlds/plank.dae'
plank = env.ReadKinBodyURI(dae15)
env.AddKinBody(plank)

In [8]:
env.Save('/home/abhyudaya/catkin_ws/src/planning/worlds/pedestal.dae')

In [4]:
plank = env.GetKinBody('plank4')

In [ ]:

model_files = json.load(open(ENV_GEN_DIR + 'model_files.json', 'r'))

def spawn(model_name, model_file, pose):
    with open(model_file, "r") as f:
        model = f.read()

    initial_pose = Pose()
    initial_pose.position.x = pose[4]
    initial_pose.position.y = pose[5]
    initial_pose.position.z = pose[6]
    initial_pose.orientation.x = pose[1]
    initial_pose.orientation.y = pose[2]
    initial_pose.orientation.z = pose[3]
    initial_pose.orientation.w = pose[0]
    
    state_msg = ModelState()
    state_msg.model_name = model_name
    state_msg.pose.position.x = pose[4]
    state_msg.pose.position.y = pose[5]
    state_msg.pose.position.z = pose[6]
    state_msg.pose.orientation.x = pose[1]
    state_msg.pose.orientation.y = pose[2]
    state_msg.pose.orientation.z = pose[3]
    state_msg.pose.orientation.w = pose[0]

    print model_name
    
    spawn_model(model_name, model, "", initial_pose, "world")
    set_state(state_msg)

# RUN `rosrun gazebo_ros gazebo`
print("Waiting for gazebo services...")
rospy.init_node("spawner")

# rospy.wait_for_service("gazebo/delete_model")
rospy.wait_for_service("gazebo/spawn_sdf_model")
rospy.wait_for_service('/gazebo/set_model_state')

set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
# delete_model = rospy.ServicePoxy("gazebo/delete_model", DeleteModel)
spawn_model = rospy.ServiceProxy("gazebo/spawn_sdf_model", SpawnModel)

for body_name in model_files.keys():
    if 'walls' in body_name or 'floor' in body_name:
        continue
    body = env.GetKinBody(body_name)
    body_pose = openravepy.poseFromMatrix(body.GetTransform())
    spawn(body_name, model_files[body_name]['sdf'], body_pose)


# ----------------- OLD REPLANNING ------------------

In [ ]:
def arm_tuck():
    joints = ["shoulder_pan_joint", "shoulder_lift_joint", "upperarm_roll_joint",
                "elbow_flex_joint", "forearm_roll_joint", "wrist_flex_joint", "wrist_roll_joint"]
    pose = [1.32, 1.40, -0.2, 1.72, 0.0, 1.66, 0.0]

    move_group = MoveGroupInterface("arm", "base_link")
    while not rospy.is_shutdown():
        result = move_group.moveToJointPosition(joints, pose, 0.02)
        if result.error_code.val == MoveItErrorCodes.SUCCESS:
            return
arm_tuck()

In [ ]:
# sms_client('fetch', [-0.08185400415346364, -9.799051334930155, 4.71238898038469]) # correct pose
sms_client('fetch', [-0.030025661256, -9.74069864204, 4.71238898038469]) # inaccurate pose

In [ ]:
res = gms_client('fetch', 'world')
current_pose = res.pose
print("fetch")
print(current_pose.position)
q = [current_pose.orientation.w,current_pose.orientation.x, current_pose.orientation.y, current_pose.orientation.z]
angle = axisAngleFromQuat(q)
print(angle)

res = gms_client('ply_object', 'base_link')
print("object rel to base")
print(res.pose.position)


In [ ]:
pose = PoseStamped()
pose.header.stamp = rospy.Time.now()
pose.header.frame_id = res.header.frame_id
pose.pose = res.pose
pose.pose.position.x -= 0.01
pose.pose.position.y += (0.32647509/2)

quaternion = tf.transformations.quaternion_from_euler(np.pi/2, 0, 0)
pose.pose.orientation.x = quaternion[0]
pose.pose.orientation.y = quaternion[1]
pose.pose.orientation.z = quaternion[2]
pose.pose.orientation.w = quaternion[3]

print(pose)

pose = PoseStamped()
pose.header.stamp = rospy.Time.now()
pose.header.frame_id = "base_link"
pose.pose.position.x = 1
pose.pose.position.y = 0
pose.pose.position.z = 1

quaternion = tf.transformations.quaternion_from_euler(np.pi/2, 0, 0)
pose.pose.orientation.x = quaternion[0]
pose.pose.orientation.y = quaternion[1]
pose.pose.orientation.z = quaternion[2]
pose.pose.orientation.w = quaternion[3]

print(pose)

In [ ]:
move_group = MoveGroupInterface("arm", "base_link")
while not rospy.is_shutdown():
    result = move_group.moveToPose(pose, 'gripper_link')
    # print(result)
    print(result.error_code.val)
    if result.error_code.val == MoveItErrorCodes.SUCCESS:
        print("success")
        break

In [ ]:
def arm_tuck():
    joints = ["shoulder_pan_joint", "shoulder_lift_joint", "upperarm_roll_joint",
                "elbow_flex_joint", "forearm_roll_joint", "wrist_flex_joint", "wrist_roll_joint"]
    pose = [1.32, 1.40, -0.2, 1.72, 0.0, 1.66, 0.0]

    move_group = MoveGroupInterface("arm", "base_link")
    while not rospy.is_shutdown():
        result = move_group.moveToJointPosition(joints, pose, 0.02)
        if result.error_code.val == MoveItErrorCodes.SUCCESS:
            return
arm_tuck()

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
g

In [ ]:
scene.

In [ ]:

group_name = "arm_with_torso"
group = moveit_commander.MoveGroupCommander(group_name)

pose_goal = Pose()
pose_goal.orientation.w = 1.0
pose_goal.position.x = 0.84
pose_goal.position.y = -0.18
pose_goal.position.z = 0.8
group.set_pose_target(pose_goal)

plan = group.go(wait=True)
print(plan)
# Calling `stop()` ensures that there is no residual movement
group.stop()
# It is always good to clear your targets after planning with poses.
# Note: there is no equivalent function for clear_joint_value_targets()
group.clear_pose_targets()

In [ ]:
init_pose = [0.0, 1.31993457, 1.40000903, -0.200000347, 1.71990992, -1.02841391e-05, 0, 7.12985942e-06]

robot.SetActiveDOFValues(init_pose)